In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

In [2]:
#if you upload a zip file to JupyterLab:
#import zipfile as zf
#files = zf.ZipFile("/home/jovyan/data.zip", 'r')
#files.extractall('/home/jovyan/')
#files.close()

In [3]:
df = pd.DataFrame()
initial = "/content/drive/MyDrive/Colab Notebooks/" 
#initial = ""
all_files = glob.glob(initial + "data/*")
course_data = glob.glob(initial + "data/uc.berkeley.student.course.data.*.txt")
majors = glob.glob(initial + "data/uc.berkeley.student.majors.*.txt")
pathways = glob.glob(initial + "data/uc.berkeley.stem.*.txt")
passes = glob.glob(initial + "data/jacobs.*")

In [4]:
# cell to check files added to variables
passes

['/content/drive/MyDrive/Colab Notebooks/data/jacobs.participants.txt']

In [5]:
stem_path = pd.read_table(pathways[0])
stem_path

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (33,43,44,50,51,53,55,56,57,58,59,60,61,62,63,64,65,66,67,72,73,74,75,76,77,78,79,80,81,82,83,89,90,92,93,94,95,96,97,98,99,100,101,102,103) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,ppsk,count,year,term,entry,residency,gender,ethnic.l1,ethnic.l2,ethnic.l3,probation.year.1,GradYr,RetainY1,RetainY2,RetainY3,RetainY4,RetainY5,RetainY6,hs.gpa.unweighted,hs.gpa.weighted,sat.total,sat.math,sat.verbal,sat.writing,act.composite,act.english,act.writing,act.math,act.reading,act.science,first.generation.college.1,first.generation.college.2,hs.api,ceeb,income.parent,income.student,major.intended.college,major.intended.division,major.intended.department,esl,...,final.major.college.2,final.major.division.2,final.major.department.2,final.major.major.2,first.major.college.3,first.major.division.3,first.major.department.3,first.major.major.3,final.major.college.3,final.major.division.3,final.major.department.3,final.major.major.3,first.major.college.4,first.major.division.4,first.major.department.4,first.major.major.4,final.major.college.4,final.major.division.4,final.major.department.4,final.major.major.4,year.degree,term.degree,academic.year.degree,gpa.cum.degree,degree.college.1,degree.division.1,degree.department.1,degree.major.1,degree.college.2,degree.division.2,degree.department.2,degree.major.2,degree.college.3,degree.division.3,degree.department.3,degree.major.3,degree.college.4,degree.division.4,degree.department.4,degree.major.4
0,1023906,1,2011,Fall,ADVANCED STANDING,Resident,Female,Underrepresented Minority,African American,African American/Black,N,3.0,Y,Y,Y,Y,Y,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,4052,NaN,26000.0,Clg of Environmental Design,Clg of Environmental Design,Landscape Arch & Env Plan,English+Other,...,NaN,NaN,NaN,NaN,Clg of Env Design,Clg of Env Design,Landscape Arch & Env Plan,Landscape Architecture,Clg of Natural Resources,Clg of Natural Resources,"Env Sci, Policy, & Mgmt",Conserv & Resource Stds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0,Spring,2013-14,3.871,Clg of Environmental Design,Clg of Environmental Design,Landscape Arch & Env Plan,Landscape Architecture,Clg of Natural Resources,Clg of Natural Resources,"Env Sci, Policy, & Mgmt",Conserv & Resource Stds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,161295,1,2015,Fall,ADVANCED STANDING,Resident,Male,White/Other,White,White,N,NaN,Y,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,7711,NaN,90000.0,Clg of Environmental Design,Clg of Environmental Design,City & Regional Planning,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,183848,1,2010,Spring,ADVANCED STANDING,Resident,Male,White/Other,White,White,N,2.0,Y,Y,Y,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,U,NaN,4061,NaN,32000.0,Clg of Letters & Science,L&S Arts & Humanities Division,Art Practice,-,...,NaN,NaN,NaN,NaN,Clg of Letters & Science,L&S-Arts & Humanities Div,Art Practice,Art Practice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,Fall,2011-12,3.977,Clg of Letters & Science,L&S Arts & Humanities Division,Art Practice,Art,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,232294,1,2009,Fall,ADVANCED STANDING,Resident,Female,White/Other,Other/Decline to State,Decline to State,N,2.5,Y,Y,Y,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,4052,NaN,39003.0,Clg of Letters & Science,L&S Arts & Humanities Division,Slavic Languages & Literatures,-,...,NaN,NaN,NaN,NaN,Clg of Letters & Science,L&S-Arts & Humanities Div,Slavic Languages & Lit,Slavic Lang & Lit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,Fall,2011-12,3.939,Clg of Letters & Science,L&S Arts & Humanities Division,Slavic Languages & Literatures,Slavic Lang & Lit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,225526,1,2010,Fall,ADVANCED STANDING,Resident,Female,White/Other,White,White,N,2.0,Y,Y,Y,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,7711,NaN,NaN,Clg of Environmental Design,Clg of Environmental Design,Architecture,English,...,NaN,NaN,NaN,NaN,Clg of Env Design,Clg of Env Design,Architecture,Architecture,NaN,NaN,NaN,NaN,Na

In [7]:
# Be aware, this cell is very resource intensive
course_data.sort()
student_courses = pd.DataFrame()
for f in course_data:
    #temp = pd.read_table(f)
    # looking at only DESINV courses
    #if(temp['year'] >= 2015):
    student_courses = student_courses.append(pd.read_table(f))

# minor data cleaning and storing student_courses
student_courses = student_courses[~student_courses['ppsk'].isnull()]
student_courses = student_courses[student_courses['ppsk'] != -1]
for i in student_courses.columns:
    if student_courses[i].isnull().all():
        student_courses.drop(i,axis=1,inplace=True)
student_courses['ppsk'] = student_courses['ppsk'].astype(int)
%store student_courses

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [33]:
majors

['/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.majors.2007.2017.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.majors.2018.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.majors.2019.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.majors.2020.txt']

In [32]:
# Be aware, this cell is very resource intensive
course_data.sort()
student_majors = pd.DataFrame()
for f in majors:
    student_courses = student_courses.append(pd.read_table(f))

# minor data cleaning and storing student_courses
student_majors = student_majors[~student_majors['ppsk'].isnull()]
student_majors = student_majors[student_majors['ppsk'] != -1]
for i in student_majors.columns:
    if student_majors[i].isnull().all():
        student_majors.drop(i,axis=1,inplace=True)
student_majors['ppsk'] = student_majors['ppsk'].astype(int)
%store student_majors

['/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.majors.2007.2017.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.majors.2018.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.majors.2019.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.majors.2020.txt']

In [29]:
# every student who holds a maker pass
maker_passes = pd.read_table(passes[0])
maker_passes = maker_passes[maker_passes['ppsk'] != 'MISSING']
#maker_passes = maker_passes['ppsk'].unique()

# all students who took design innovation course
jacobs = student_courses[student_courses["course.department"] == "Design Innovation"]

In [30]:
jacobs['ppsk']

164       974383
605      1474045
1910     2418191
2380     2420051
2915     2423832
          ...   
21309    4262284
21441    4262621
22125    4264306
22572    4265873
22697    4266391
Name: ppsk, Length: 783, dtype: int64

In [11]:
#temp = student_courses[student_courses['year.name'].str.split(" ", expand=True)[0].astype(int) >= 2015]

In [31]:
ppsks = maker_passes['ppsk']
ppsks = ppsks.append(jacobs['ppsk']).unique()

In [28]:
all_jacobs = student_courses

9096

In [13]:
len(maker_passes['ppsk'].unique())

8449

In [14]:
student_courses[student_courses["course.subject"] == "Design Innovation"]['year.name'].unique()

array(['2015 Fall', '2015 Spring', '2016 Spring', '2016 Fall',
       '2017 Spring', '2017 Summer', '2017 Fall', '2018 Fall',
       '2018 Spring', '2018 Summer', '2019 Fall', '2019 Spring',
       '2019 Summer'], dtype=object)

In [15]:
student_courses['year.name']#.query('year >= 2015')

93       2007 Summer
94       2007 Summer
95       2007 Summer
96       2007 Summer
97       2007 Summer
            ...     
23102    2019 Summer
23103    2019 Summer
23104    2019 Summer
23105    2019 Summer
23106    2019 Summer
Name: year.name, Length: 5597609, dtype: object

In [17]:
student_courses[student_courses[['grade.subtype', 'grade.type', 'grade.name']]['grade.type'] == 'Non-Letter Grade']['grade.name'].unique()

array(['High Honors', 'Honors', 'Pass', 'Pass Conditional', 'Credit',
       'Satisfactory', 'No Credit', 'Not Pass', 'Unsatisfactory'],
      dtype=object)